# Dash Testing

In [4]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import numpy as np
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import plotly.express as px

from dash import Dash, dcc, html, Input, Output, callback

In [5]:
# Setup the transport and client
_transport = RequestsHTTPTransport(
    url='http://127.0.0.1:8000/graphql/',
    use_json=True,
)

client = Client(
    transport = _transport,
    fetch_schema_from_transport=True,
)

# Define Query
query = gql('''
query playerPointTotals {
  playerTotalsAll(limit: 35, ordering: "-points") {
    playerName
    position
    team
    season
    minutesPg
    points
    id
  }
}
''')

# Execute and store response

response = client.execute(query)

print(response)

{'playerTotalsAll': [{'playerName': 'Kobe Bryant*', 'position': 'SG', 'team': 'LAL', 'season': 2006, 'minutesPg': '3277.0', 'points': 2832, 'id': '10755'}, {'playerName': 'James Harden', 'position': 'PG', 'team': 'HOU', 'season': 2019, 'minutesPg': '2867.0', 'points': 2818, 'id': '3118'}, {'playerName': 'Kevin Durant', 'position': 'SF', 'team': 'OKC', 'season': 2014, 'minutesPg': '3122.0', 'points': 2593, 'id': '6229'}, {'playerName': 'Russell Westbrook', 'position': 'PG', 'team': 'OKC', 'season': 2017, 'minutesPg': '2802.0', 'points': 2558, 'id': '4778'}, {'playerName': 'Michael Jordan*', 'position': 'SG', 'team': 'CHI', 'season': 1993, 'minutesPg': '3067.0', 'points': 2541, 'id': '17728'}, {'playerName': 'Michael Jordan*', 'position': 'SG', 'team': 'CHI', 'season': 1996, 'minutesPg': '3090.0', 'points': 2491, 'id': '16287'}, {'playerName': 'LeBron James', 'position': 'SF', 'team': 'CLE', 'season': 2006, 'minutesPg': '3361.0', 'points': 2478, 'id': '10931'}, {'playerName': 'Kevin Dura

In [9]:
app = Dash(__name__)

df= pd.DataFrame(response['playerTotalsAll'])
df['minutesPg'] = df['minutesPg'].astype(float)

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='xaxis-column',
                options=[{'label': i, 'value': i} for i in df.columns],
                value='minutesPg'
            ),
            dcc.RadioItems(
                id='xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                inline=True
            )
        ], style={'width': '48%', 'display': 'inline-block'}),
        
        html.Div([
            dcc.Dropdown(
                id='yaxis-column',
                options=[{'label': i, 'value': i} for i in df.columns],
                value='points'
            ),
            dcc.RadioItems(
                id='yaxis-type',
                inline=True,
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear'
            )
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),
        
    ]),

    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        min=df['season'].min(),
        max=df['season'].max(),
        step=None,
        id='season-slider',
        value=df['season'].max(),
        marks={str(season): str(season) for season in df['season'].unique()},
    )
])


@callback(
    Output('indicator-graphic', 'figure'),
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'),
    Input('xaxis-type', 'value'),
    Input('yaxis-type', 'value'),
    Input('season-slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 season_value):
    dff = df[df['season'] == season_value]

    fig = px.scatter(dff, x=xaxis_column_name, y=yaxis_column_name, hover_name='playerName')

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    fig.update_xaxes(title=xaxis_column_name,
                     type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name,
                     type='linear' if yaxis_type == 'Linear' else 'log')

    return fig


if __name__ == '__main__':
    app.run(debug=True)

